In [51]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os


data_folder = '/data'
data = []


print(os.path.curdir)
#for filenames in os.open(data_folder, ):
#    data.append(filenames)


data


.


[]

### Data check and organization

In [26]:
train_path, test_path, gender_submission_path = data


train = pd.read_csv(train_path)

test = pd.read_csv(test_path)


ValueError: not enough values to unpack (expected 3, got 0)

### Finding and filling nan values

In [184]:
train['Age'].isna().sum()
train['Cabin'].isna().sum()
train['Embarked'].isna().sum()

#print(train['Age'].value_counts(dropna=False))
train['Age'].describe()
#Age mean is 29.699118 so i'll replace all nan's in Age column with this value,
#don't know if this is the right move, buti know it's better than zero

train['Cabin'].value_counts(dropna=False)
#still don't know what to do with this one theres a lot of nans 687 out of 891
#as f right now i'll just ignore this column even tho it seems important.
 
train['Embarked'].value_counts(dropna=False)
#theres only 2 nans so im probably gonna drop the records, it's nt a big loss
train = train.dropna(subset=['Embarked'])

train = train.fillna({'Age':29.699118})

### X and Y extraction

In [185]:
y = train['Survived']

# all columns
# ['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']

# Ignoring Cabin, Name, Ticket, Fare
x_columns = ['PassengerId','Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Embarked']

X_pre_encoding = train[x_columns]
X_pre_encoding.isna().sum()
X_pre_encoding.count()

PassengerId    889
Pclass         889
Sex            889
Age            889
SibSp          889
Parch          889
Embarked       889
dtype: int64

### X one hot encoding

In [186]:
X = pd.get_dummies(X_pre_encoding, columns=['Sex', 'Embarked'])

X

,PassengerId,Pclass,Age,SibSp,Parch,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,1,3,22.000000,1,0,False,True,False,False,True
1,2,1,38.000000,1,0,True,False,True,False,False
2,3,3,26.000000,0,0,True,False,False,False,True
3,4,1,35.000000,1,0,True,False,False,False,True
4,5,3,35.000000,0,0,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...
886,887,2,27.000000,0,0,False,True,False,False,True
887,888,1,19.000000,0,0,True,False,False,False,True
888,889,3,29.699118,1,2,True,False,False,False,True
889,890,1,26.000000,0,0,False,True,True,False,False


### Creating model and fitting data

In [187]:
from sklearn.tree import DecisionTreeRegressor

model = DecisionTreeRegressor(random_state = 1)

model.fit(X, y)

DecisionTreeRegressor(random_state=1)

### testing the model

In [194]:
test1 = pd.get_dummies(test, columns=['Sex', 'Embarked'])

test1 = test.drop(columns=['Name', 'Ticket', 'Fare', 'Cabin'])

print(model.predict(test1))


KeyError: "None of [Index(['Sex', 'Embarked'], dtype='object')] are in the [columns]"